In [176]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np

In [ ]:
import sys
root_dir = "\\".join(sys.path[0].split("\\")[:-2])

In [38]:
def save_obj(obj, name):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
KG = nx.read_gpickle(root_dir+"\\data\\graph\\training_KG_concepts.gpickle")

In [4]:
all_concepts = list(KG.nodes())
all_concepts_edges = list(KG.edges())

In [11]:
edges_pc = pd.read_csv(root_dir+"\\data\\og\\edges_pc.csv")[["dst", "src"]]
papers = pd.read_csv(root_dir+"\\data\\og\\papers.csv")
papers = papers.fillna(0)
papers["yearmonth"] = papers["year"]*12+papers["month"]

In [27]:
papers_dict = {}
for i in range(len(papers)):
    paper_id = papers.id[i]
    papers_dict[paper_id] = papers.yearmonth[i]

In [33]:
concepts_dates = {}
for i in range(len(edges_pc)):
    concept = edges_pc.dst[i]
    paper = edges_pc.src[i]
    if concept in concepts_dates:
        concepts_dates[concept].append(
            papers_dict[str(paper)])
    else:
        concepts_dates[concept] = [
            papers_dict[str(paper)]]
save_obj(concepts_dates, "concepts_dates")

In [50]:
concepts_last_dates = {}
for concept in all_concepts:
    if concept not in concepts_dates:
        continue
    concepts_last_dates[concept] = max(concepts_dates[concept])
save_obj(concepts_last_dates, "concepts_last_dates")

In [53]:
june_2020 = 2020*12+6

Collect training and validation papers

In [61]:
all_concepts_edges = list(KG.edges())
all_concepts_edges = [
    i for i \
    in all_concepts_edges \
    if i[0]==i[0] and i[1]==i[1]
]

In [ ]:
training_papers = [
    paper for paper \
    in papers_dict \
    if papers_dict[paper]<=june_2020
]
validation_papers = [
    paper for paper \
    in papers_dict \
    if papers_dict[paper]>june_2020
]
save_obj(training_papers, "training_papers")
save_obj(validation_papers, "validation_papers")

Collect training and validation edges

In [ ]:
training_edges_pc = edges_pc[edges_pc["src"].isin(training_papers)]

In [171]:
training_concepts = list(set(training_edges_pc.dst.values.tolist()))
training_papers = list(set(training_edges_pc.src.values.tolist()))

In [ ]:
training_edges = []
validation_edges = []
for i in range(len(all_concepts_edges)):
    edge = all_concepts_edges[i]
    concept0_papers = training_edges_pc[
        (training_edges_pc["dst"]==edge[0])
    ].src
    concept1_papers = training_edges_pc[
        (training_edges_pc["dst"]==edge[1])
    ].src
    common_papers = set(concept0_papers).intersection(set(concept1_papers))
    if common_papers:
        training_edges.append(edge)
    else:
        validation_edges.append(edge)

In [201]:
save_obj(training_edges, "training_edges")
save_obj(validation_edges, "validation_edges")

In [208]:
training_concepts = list(set(
    [i[0] for i in training_edges] \
    +[i[1] for i in training_edges]
))
validation_concepts = list(set(
    [i[0] for i in validation_edges] \
    +[i[1] for i in validation_edges]
))

save_obj(training_concepts, "training_concepts")
save_obj(validation_concepts, "validation_concepts")

Create Training Concept Graph

In [212]:
training_KG = nx.Graph()

In [213]:
for concept in training_concepts:
    if concept == concept:
        training_KG.add_node(concept, node_type = "concept")

In [214]:
training_KG.add_edges_from(training_edges)

In [219]:
print("Edges: ", len(list(training_KG.edges(data=True))))
print("Nodes: ", len(list(training_KG.nodes(data=True))))

Edges:  300673
Nodes:  46669


In [220]:
nx.write_gpickle(training_KG, "training_KG_concepts.gpickle")
#KG = nx.read_gpickle("training_KG_concepts.gpickle")